# Bibliography Categorization: 'BibCat'
## Tutorial: Estimating performance of classifiers in bibcat.



---


## Introduction.

In this tutorial, we will use bibcat to estimate the performance of classifiers on sets of texts.


---

## User Workflow: Training a machine learning (ML) model.


The `Performance` class contains user-friendly methods for estimating the performance of given classifiers and outputting that performance as, e.g., confusion matrices.  We overview how this method can be run in the code blocks below.

For this tutorial, we assume that the user has already run the trainML tutorial, and so has generated and saved a machine learning model.

In [1]:
#Import external packages
import re
import os
import sys
import json
import numpy as np


In [2]:
# Set up for fetching necessary bibcat modules for the tutorial
# Check work directories: src/ is where all source python scripts are available. 
current_dir= os.path.dirname(os.path.abspath('__file__'))
_parent = os.path.dirname(current_dir)
src_dir = os.path.join(_parent, "src")

print(f'Current Directory: {current_dir}')
print(f'Source directory: {src_dir}')

# move to the ../src/ directory to import necessary modules. 
os.chdir(src_dir)

Current Directory: /Users/jamila.pegues/Documents/STScI_Fellowship/Functional/Library/BibTracking/repo_stsci/bibcat/docs
Source directory: /Users/jamila.pegues/Documents/STScI_Fellowship/Functional/Library/BibTracking/repo_stsci/bibcat/src


In [3]:
#Import bibcat packages
import bibcat_classes as bibcat
import bibcat_config as config
import bibcat_constants as preset
import bibcat_parameters as params #Temporary file until contents moved elsewhere

Root directory =/Users/jamila.pegues/Documents/STScI_Fellowship/Functional/Library/BibTracking/repo_stsci/bibcat/src, parent directory=/Users/jamila.pegues/Documents/STScI_Fellowship/Functional/Library/BibTracking/repo_stsci/bibcat
/Users/jamila.pegues/Documents/STScI_Fellowship/Functional/Library/BibTracking/repo_stsci/bibcat/src/models folder already exists.
/Users/jamila.pegues/Documents/STScI_Fellowship/Functional/Library/BibTracking/repo_stsci/bibcat/output folder already exists.


In [4]:
#Set parameters for each operator and its internal classifier
#Global parameters
do_verify_truematch = False #A very important parameter!!!  Set it to either True or False.
do_raise_innererror = False #If True, will stop if exception encountered; if False, will print error and continue
list_threshold_arrays = [np.linspace(0.5, 0.95, 20)]*2 #For uncertainty test
class_mapper = params.map_papertypes #Mapper for class types; None for no mapper

#For operator 1
mapper_1 = class_mapper #Mapper to mask classifications; None if no masking
threshold_1 = 0.70 #Uncertainty threshold for this classifier
buffer_1 = 0

#For operator 2
mapper_2 = class_mapper #Mapper to mask classifications; None if no masking
threshold_2 = 0.70 #Uncertainty threshold for this classifier
buffer_2 = 0

#Gather parameters into lists
list_mappers = [mapper_1, mapper_2]
list_thresholds = [threshold_1, threshold_2]
list_buffers = [buffer_1, buffer_2]

In [5]:
#Set some overarching global variables
seed_test = 10 #Random seed for shuffling text dataset
np.random.seed(seed_test)
do_shuffle = True #Whether or not to shuffle the text dataset
do_real_testdata = True #If True, will use real papers to test performance; if False, will use fake texts below
#
max_tests = 100 #Number of text entries to test the performance for; None for all tests available
#is_text_processed = False #We are using preprocessed text for this tutorial (previously generated by trainML in a test set directory)
mode_modif = "skim_trim_anon" #None #We are using preprocessed data in this tutorial, so we do not need a processing mode at all
#buffer = 0
#
#
#Prepare some Keyword objects
#kobj_hubble = bibcat.Keyword(
#                keywords=["Hubble", "Hubble Telescope",
#                          "Hubble Space Telescope"],
#                acronyms=["hst", "ht"])
all_kobjs = params.all_kobjs #[kobj_hubble]

In [6]:
#Fetch filepaths for model and data
name_model = config.name_model
filepath_json = config.path_json
output_path = config.PATH_OUTPUT
dir_model = os.path.join(config.dir_allmodels, name_model)
#
#Set directories for storing performance output
filepath_output = output_path #Where to store the performance output, such as the confusion matrices
#
#Set directories for fetching text
dir_info = dir_model
folder_test = preset.folders_TVT["test"]
dir_test = os.path.join(dir_model, folder_test)

Let's build a set of classifiers for which we'd like to test the performance.  We'll then feed each of those classifiers into an instance of the Operator class to handle them.

In [7]:
#Create a list of classifiers
#This can be modified to use whatever classifiers you'd like.
#Load a previously trained ML model
filepath_model = os.path.join(dir_model, (name_model+".npy"))
fileloc_ML = os.path.join(dir_model, (preset.tfoutput_prefix+name_model))
classifier_ML = bibcat.Classifier_ML(filepath_model=filepath_model, fileloc_ML=fileloc_ML, do_verbose=True)
#

#Load a rule-based classifier
classifier_rules = bibcat.Classifier_Rules()
#

INFO:absl:using Lamb optimizer


In [8]:
#Load models into instances of the Operator class
operator_1 = bibcat.Operator(classifier=classifier_ML, mode=mode_modif, keyword_objs=all_kobjs,
                            name="Operator_1", do_verbose=True, load_check_truematch=True, do_verbose_deep=False)
operator_2 = bibcat.Operator(classifier=classifier_rules,
                            name="Operator_2", mode=mode_modif, keyword_objs=all_kobjs, do_verbose=True, do_verbose_deep=False)
list_operators = [operator_1, operator_2] #Feel free to add more/less operators here.
#

Instance of Operator successfully initialized!
Keyword objects:
0: Keyword Object:
Name: Hubble
Keywords: ['Hubble Space Telescope', 'Hubble Telescope', 'Hubble']
Acronyms: ['HST', 'HT']

1: Keyword Object:
Name: Webb Telescope
Keywords: ['James Webb Space Telescope', 'Webb Space Telescope', 'James Webb Telescope', 'Webb Telescope']
Acronyms: ['JWST', 'JST', 'JT']

2: Keyword Object:
Name: Transiting Exoplanet Survey Satellite
Keywords: ['Transiting Exoplanet Survey Satellite']
Acronyms: ['TESS']

3: Keyword Object:
Name: KEPLER
Keywords: ['KEPLER']
Acronyms: []

4: Keyword Object:
Name: Pan-STARRS
Keywords: ['Panoramic Survey Telescope and Rapid Response System', 'Pan-STARRS1', 'Pan-STARRS']
Acronyms: ['PanSTARRS1', 'PanSTARRS', 'PS1']

5: Keyword Object:
Name: Galaxy Evolution Explorer
Keywords: ['Galaxy Evolution Explorer']
Acronyms: ['GALEX']

6: Keyword Object:
Name: K2
Keywords: ['K2']
Acronyms: []

7: Keyword Object:
Name: Hubble Legacy Archive
Keywords: ['Hubble Legacy Archive'

Now, let's fetch some text for our classifiers to classify. For this tutorial, we'll load previously processed texts from the directory containing the test set for the ML classifier.

In [9]:
#For use of real papers from test dataset to test on
if (do_real_testdata):
    #Load information for processed bibcodes reserved for testing
    dict_TVTinfo = np.load(os.path.join(dir_info, "dict_TVTinfo.npy"), allow_pickle=True).item()
    list_test_bibcodes = [key for key in dict_TVTinfo if (dict_TVTinfo[key]["folder_TVT"] == folder_test)]
    
    #Load the original data
    with open(filepath_json, 'r') as openfile:
        dataset = json.load(openfile)
    #
    
    #Extract text information for the bibcodes reserved for testing
    list_test_indanddata_raw = [(ii, dataset[ii]) for ii in range(0, len(dataset))
                                if (dataset[ii]["bibcode"] in list_test_bibcodes)] #Data for test set
    #
    
    #Shuffle, if requested
    if do_shuffle:
        np.random.shuffle(list_test_indanddata_raw)
    #
    
    #Extract target number of test papers from the test bibcodes
    if (max_tests is not None): #Fetch subset of tests
        list_test_indanddata = list_test_indanddata_raw[0:max_tests]
    else: #Use all tests
        list_test_indanddata = list_test_indanddata_raw
    #
    
    #Process the text input into dictionary format for inputting into the codebase
    dict_texts = {} #To hold formatted text entries
    for ii in range(0, len(list_test_indanddata)):
        curr_ind = list_test_indanddata[ii][0]
        curr_data = list_test_indanddata[ii][1]
        #
        #Convert this data entry into dictionary with: key:text,id,bibcode,mission structure
        curr_info = {"text":curr_data["body"], "id":str(curr_ind), "bibcode":curr_data["bibcode"],
                    "missions":{}}
        for curr_mission in curr_data["class_missions"]: #Iterate through missions for this paper
            for curr_kobj in all_kobjs: #Iterate through declared Keyword objects
                curr_name = curr_kobj.get_name()
                #Store mission data under keyword name, if applicable
                if (curr_kobj.is_keyword(curr_mission)):
                    curr_info["missions"][curr_name] = {"mission":curr_name,
                                                    "class":curr_data["class_missions"][curr_mission]["papertype"]}
                #
                #Otherwise, store that this mission was not detected for this text
                else:
                    curr_info["missions"][curr_name] = {"mission":curr_name, "class":preset.verdict_rejection}                    
            #
        #
        #Store this data entry
        dict_texts[str(curr_ind)] = curr_info
    #
    
    #Print some notes about the testing data
    print("Number of texts in text set: {0}".format(len(dict_texts)))
    print("")
    for key in dict_texts:
        print("Entry {0}:".format(key))
        print("ID: {0}".format(dict_texts[key]["id"]))
        print("Bibcode: {0}".format(dict_texts[key]["bibcode"]))
        print("Missions: {0}".format(dict_texts[key]["missions"]))
        print("Start of text:\n{0}".format(dict_texts[key]["text"][0:500]))
        print("-\n")
    #
#

Number of texts in text set: 100

Entry 106:
ID: 106
Bibcode: 2021MNRAS.500.1491T
Missions: {'Hubble': {'mission': 'Hubble', 'class': 'z_notmatch'}, 'Webb Telescope': {'mission': 'Webb Telescope', 'class': 'z_notmatch'}, 'Transiting Exoplanet Survey Satellite': {'mission': 'Transiting Exoplanet Survey Satellite', 'class': 'z_notmatch'}, 'KEPLER': {'mission': 'KEPLER', 'class': 'z_notmatch'}, 'Pan-STARRS': {'mission': 'Pan-STARRS', 'class': 'z_notmatch'}, 'Galaxy Evolution Explorer': {'mission': 'Galaxy Evolution Explorer', 'class': 'z_notmatch'}, 'K2': {'mission': 'K2', 'class': 'z_notmatch'}, 'Hubble Legacy Archive': {'mission': 'Hubble Legacy Archive', 'class': 'SCIENCE'}}
Start of text:
1 INTRODUCTION An active galactic nucleus (AGN) is a compact region at the centre of a galaxy that emits a significant amount of energy over much of the electromagnetic spectrum, and whose spectral characteristics indicate that the energy source is non-stellar. Such objects have been observed in the 

In [10]:
#For use of fake, made-up data entries to test on
if (not do_real_testdata):
    print("Using fake test data for testing.")
    #Make some fake data
    dict_texts_raw = {"science":["We present HST observations in Figure 4.",
                        "The HST stars are listed in Table 3b.",
                        "Despite our efforts to smooth the data, there are still rings in the HST images.",
                        "See Section 8c for more discussion of the Hubble images.",
                        "The supernovae detected with HST tend to be brighter than initially predicted.",
                        "Our spectra from HST fit well to the standard trend first published in Someone et al. 1990.",
                        "We use the Hubble Space Telescope to build an ultraviolet database of the target stars.",
                        "The blue points (HST) exhibit more scatter than the red points (JWST).",
                        "The benefit, then, is the far higher S/N we achieved in our HST observations.",
                        "Here we employ the Hubble Telescope to observe the edge of the photon-dominated region.",
                        "The black line shows that the region targeted with Hubble has an extreme UV signature."],
                 "datainfluenced":["The simulated Hubble data is plotted in Figure 4.",
                       "Compared to the HST observations in Someone et al., our JWST follow-up reached higher S/N.",
                       "We were able to reproduce the luminosities from Hubble using our latest models.",
                       "We overplot Hubble-observed stars from Someone et al. in Figure 3b.",
                       "We built the spectral templates using UV data in the Hubble archive.",
                       "We simulate what our future HST observations will look like to predict the S/N.",
                       "Our work here with JWST is inspired by our earlier HST study published in 2010.",
                       "We therefore use the Hubble statistics from Author et al. to guide our stellar predictions.",
                       "The stars in Figure 3 were plotted based on the HST-fitted trend line in Person et al.",
                       "The final step is to use the HST exposure tool to put our modeled images in context."],
                 "mention":["Person et al. used HST to measure the Hubble constant.",
                        "We will present new HST observations in a future work.",
                        "HST is a fantastic instrument that has revolutionized our view of space.",
                        "The Hubble Space Telescope (HST) has its mission center at the STScI.",
                        "We can use HST to power a variety of science in the ultraviolet regime.",
                        "It is not clear when the star will be observable with HST.",
                        "More data can be found and downloaded from the Hubble archive.",
                        "We note that HST can be used to observe the stars as well, at higher S/N.",
                        "However, we ended up using the JWST rather than HST observations in this work.",
                        "We push the analysis of the Hubble component of the dataset to a future study.",
                        "We expect the HST observations to be released in the fall.",
                        "We look forward to any follow-up studies with, e.g., the Hubble Telescope."]}
    #
    #Convert into dictionary with: key:text,class,id,mission structure
    i_track = 0
    dict_texts = {}
    #Store subheadings by mission, to avoid duplicating and processing the same text across different missions
    mission = operator_1._fetch_keyword_object(lookup="HST")._get_info("name")
    for key in dict_texts_raw:
        curr_set = dict_texts_raw[key]
        for ii in range(0, len(curr_set)):
            dict_texts[str(i_track)] = {"text":curr_set[ii], "id":"{0}_{1}".format(key, ii), "bibcode":str(i_track),
                                        "missions":{mission:{"mission":mission, "class":key}}}
            i_track += 1
    #
    print("Mission: {0}".format(mission))
    print("Number of texts in text set: {0}".format(len(dict_texts)))
    print("")
    for key in dict_texts:
        print(dict_texts[key])
        print("-")
    #
#

Next, let's prepare some additional information for each of these classifiers.  We'll need to set, for example, the uncertainty thresholds for accepting or rejecting each classifier's output.

In [11]:
#Store texts for each operator and its internal classifier
#For operator 1
dict_texts_1 = dict_texts #Dictionary of texts to classify

#For operator 2
dict_texts_2 = dict_texts #Dictionary of texts to classify

#Gather into list
list_dict_texts = [dict_texts_1, dict_texts_2]

Now, let's evaluate the performance of these classifiers in different ways.  We will consider these performance tests:
* Basic: We generate confusion matrices for the set of Operators (containing the different classifiers).
* Uncertainty: We plot performance as a function of uncertainty level for the set of Operators.

In [12]:
#Create an instance of the Performance class
performer = bibcat.Performance()

The Basic evaluation:

In [13]:
#Parameters for this evaluation
fileroot_evaluation = "test_eval_basic" #Root name of the file within which to store the performance evaluation output
fileroot_misclassif = "test_misclassif_basic" #Root name of the file within which to store misclassified text information
figsize = (20, 12)

#Run the pipeline for a basic evaluation of model performance
performer.evaluate_performance_basic(operators=list_operators, dicts_texts=list_dict_texts, mappers=list_mappers,
                                     thresholds=list_thresholds, buffers=list_buffers, is_text_processed=False,
                                     do_verify_truematch=do_verify_truematch, do_raise_innererror=do_raise_innererror,
                                     do_save_evaluation=True, do_save_misclassif=True, filepath_output=filepath_output,
                                     fileroot_evaluation=fileroot_evaluation, fileroot_misclassif=fileroot_misclassif,
                                     print_freq=25, do_verbose=True, do_verbose_deep=False, figsize=figsize)


> Running evaluate_performance_basic()!
Generating evaluation for the given operators...

> Running _generate_evaluation()!
Iterating through Operators to classify each set of text...
Classifying with Operator #0...
Classification complete for Operator #0.
Generating the performance counter...

> Running _generate_performance_counter()!
Accumulating performance over 100 texts.
Actual class names: ['science', 'mention', 'datainfluenced', 'other', 'znotmatch']
Measured class names: ['science', 'mention', 'datainfluenced', 'other', 'zerror', 'zlowprob', 'znotmatch']

-
Performance counter generated:
Actual science total: 30
Actual science vs Measured science: 0
Actual science vs Measured mention: 0
Actual science vs Measured datainfluenced: 0
Actual science vs Measured other: 0
Actual science vs Measured zerror: 0
Actual science vs Measured zlowprob: 30
Actual science vs Measured znotmatch: 0
Actual science vs Measured _total: 30
Actual mention total: 60
Actual mention vs Measured scienc

/Users/jamila.pegues/Documents/STScI_Fellowship/Functional/Library/BibTracking/repo_stsci/bibcat/src/bibcat_classes.py:5592: RuntimeWarning: divide by zero encountered in double_scalars
  (np.abs(tmp_unnorm - curr_scores[other_key])


-
The following err. was encountered in operate:
ValueError("Err: Nothing important:\nNumber of clockwise and counterclockwise galaxies in the COSMOS field and in the field of SDSS and Pan-STARRS centred around COSMOS.\n\n{0: {'i_verb': 0, 'verb': 'Number', 'is_important': False, 'i_postverbs': [], 'i_branchwords_all': [1, 2, 3, 4, 5, 6, 9, 10, 11, 13, 14, 20, 15, 16, 17, 18, 19, 21, 22, 23], 'verbtype': []}}\n\n{0: {'i_clausechain': None, 'i_clausetrail': None, 'word': 'Number', 'index': 0, '_dep': 'ROOT', '_pos': 'NOUN', '_tag': 'NN', 'wordchunk': array(['Number'], dtype='<U6'), 'sentence': 'Number of clockwise and counterclockwise galaxies in the COSMOS field and in the field of SDSS and Pan-STARRS centred around COSMOS.', 'is_important': False, 'dict_importance': None, 'is_useless': False, 'pos_main': None}, 1: {'i_clausechain': None, 'i_clausetrail': None, 'word': 'of', 'index': 1, '_dep': 'prep', '_pos': 'ADP', '_tag': 'IN', 'wordchunk': array(['of'], dtype='<U2'), 'sentence': 'N

-
The following err. was encountered in operate:
ValueError("Err: Nothing important:\nTopping, Pan-STARRS, J.\n\n{0: {'i_verb': 0, 'verb': 'Topping', 'is_important': False, 'i_postverbs': [], 'i_branchwords_all': [1, 2, 3, 4, 5, 6], 'verbtype': ['PRESENT']}}\n\n{0: {'i_clausechain': [0], 'i_clausetrail': [0, 1, 2, 3, 4, 5, 6], 'word': 'Topping', 'index': 0, '_dep': 'ROOT', '_pos': 'VERB', '_tag': 'VBG', 'wordchunk': array(['Topping'], dtype='<U7'), 'sentence': 'Topping, Pan-STARRS, J.', 'is_important': False, 'dict_importance': None, 'is_useless': False, 'pos_main': 'VERB'}, 1: {'i_clausechain': None, 'i_clausetrail': None, 'word': ',', 'index': 1, '_dep': 'punct', '_pos': 'PUNCT', '_tag': ',', 'wordchunk': array([','], dtype='<U1'), 'sentence': 'Topping, Pan-STARRS, J.', 'is_important': False, 'dict_importance': None, 'is_useless': False, 'pos_main': None}, 2: {'i_clausechain': None, 'i_clausetrail': None, 'word': 'Pan', 'index': 2, '_dep': 'npadvmod', '_pos': 'PROPN', '_tag': 'NNP', 

Classification complete for Operator #1.
Generating the performance counter...

> Running _generate_performance_counter()!
Accumulating performance over 100 texts.
Actual class names: ['science', 'mention', 'datainfluenced', 'other', 'znotmatch']
Measured class names: ['science', 'mention', 'datainfluenced', 'other', 'zerror', 'zlowprob', 'znotmatch']

-
Performance counter generated:
Actual science total: 30
Actual science vs Measured science: 12
Actual science vs Measured mention: 1
Actual science vs Measured datainfluenced: 0
Actual science vs Measured other: 0
Actual science vs Measured zerror: 1
Actual science vs Measured zlowprob: 16
Actual science vs Measured znotmatch: 0
Actual science vs Measured _total: 30
Actual mention total: 60
Actual mention vs Measured science: 2
Actual mention vs Measured mention: 21
Actual mention vs Measured datainfluenced: 0
Actual mention vs Measured other: 0
Actual mention vs Measured zerror: 4
Actual mention vs Measured zlowprob: 31
Actual mention

The Uncertainty evaluation:

In [14]:
#Parameters for this evaluation
fileroot_evaluation = "test_eval_uncertainty" #Root name of the file within which to store the performance evaluation output
fileroot_misclassif = "test_misclassif_uncertainty" #Root name of the file within which to store misclassified text information
figsize = (40, 12)

#Run the pipeline for an evaluation of model performance as a function of uncertainty
performer.evaluate_performance_uncertainty(operators=list_operators, dicts_texts=list_dict_texts, mappers=list_mappers,
                                     threshold_arrays=list_threshold_arrays, buffers=list_buffers,
                                     is_text_processed=False,
                                     do_verify_truematch=do_verify_truematch, do_raise_innererror=do_raise_innererror,
                                     do_save_evaluation=True, do_save_misclassif=True, filepath_output=filepath_output,
                                     fileroot_evaluation=fileroot_evaluation, fileroot_misclassif=fileroot_misclassif,
                                     print_freq=25, do_verbose=True, do_verbose_deep=False, figsize=figsize)


> Running evaluate_performance_uncertainty()!
Generating evaluations for operators and uncertainties...

> Running _generate_evaluation()!
Iterating through Operators to classify each set of text...
Classifying with Operator #0...
Classification complete for Operator #0.
Generating the performance counter...

> Running _generate_performance_counter()!
Accumulating performance over 100 texts.
Actual class names: ['science', 'mention', 'datainfluenced', 'other', 'znotmatch']
Measured class names: ['science', 'mention', 'datainfluenced', 'other', 'zerror', 'zlowprob', 'znotmatch']

-
Performance counter generated:
Actual science total: 30
Actual science vs Measured science: 0
Actual science vs Measured mention: 0
Actual science vs Measured datainfluenced: 25
Actual science vs Measured other: 0
Actual science vs Measured zerror: 0
Actual science vs Measured zlowprob: 5
Actual science vs Measured znotmatch: 0
Actual science vs Measured _total: 30
Actual mention total: 60
Actual mention vs 

-
The following err. was encountered in operate:
ValueError("Err: Nothing important:\nNumber of clockwise and counterclockwise galaxies in the COSMOS field and in the field of SDSS and Pan-STARRS centred around COSMOS.\n\n{0: {'i_verb': 0, 'verb': 'Number', 'is_important': False, 'i_postverbs': [], 'i_branchwords_all': [1, 2, 3, 4, 5, 6, 9, 10, 11, 13, 14, 20, 15, 16, 17, 18, 19, 21, 22, 23], 'verbtype': []}}\n\n{0: {'i_clausechain': None, 'i_clausetrail': None, 'word': 'Number', 'index': 0, '_dep': 'ROOT', '_pos': 'NOUN', '_tag': 'NN', 'wordchunk': array(['Number'], dtype='<U6'), 'sentence': 'Number of clockwise and counterclockwise galaxies in the COSMOS field and in the field of SDSS and Pan-STARRS centred around COSMOS.', 'is_important': False, 'dict_importance': None, 'is_useless': False, 'pos_main': None}, 1: {'i_clausechain': None, 'i_clausetrail': None, 'word': 'of', 'index': 1, '_dep': 'prep', '_pos': 'ADP', '_tag': 'IN', 'wordchunk': array(['of'], dtype='<U2'), 'sentence': 'N

-
The following err. was encountered in operate:
ValueError("Err: Nothing important:\nTopping, Pan-STARRS, J.\n\n{0: {'i_verb': 0, 'verb': 'Topping', 'is_important': False, 'i_postverbs': [], 'i_branchwords_all': [1, 2, 3, 4, 5, 6], 'verbtype': ['PRESENT']}}\n\n{0: {'i_clausechain': [0], 'i_clausetrail': [0, 1, 2, 3, 4, 5, 6], 'word': 'Topping', 'index': 0, '_dep': 'ROOT', '_pos': 'VERB', '_tag': 'VBG', 'wordchunk': array(['Topping'], dtype='<U7'), 'sentence': 'Topping, Pan-STARRS, J.', 'is_important': False, 'dict_importance': None, 'is_useless': False, 'pos_main': 'VERB'}, 1: {'i_clausechain': None, 'i_clausetrail': None, 'word': ',', 'index': 1, '_dep': 'punct', '_pos': 'PUNCT', '_tag': ',', 'wordchunk': array([','], dtype='<U1'), 'sentence': 'Topping, Pan-STARRS, J.', 'is_important': False, 'dict_importance': None, 'is_useless': False, 'pos_main': None}, 2: {'i_clausechain': None, 'i_clausetrail': None, 'word': 'Pan', 'index': 2, '_dep': 'npadvmod', '_pos': 'PROPN', '_tag': 'NNP', 

Classification complete for Operator #1.
Generating the performance counter...

> Running _generate_performance_counter()!
Accumulating performance over 100 texts.
Actual class names: ['science', 'mention', 'datainfluenced', 'other', 'znotmatch']
Measured class names: ['science', 'mention', 'datainfluenced', 'other', 'zerror', 'zlowprob', 'znotmatch']

-
Performance counter generated:
Actual science total: 30
Actual science vs Measured science: 5
Actual science vs Measured mention: 6
Actual science vs Measured datainfluenced: 0
Actual science vs Measured other: 0
Actual science vs Measured zerror: 1
Actual science vs Measured zlowprob: 18
Actual science vs Measured znotmatch: 0
Actual science vs Measured _total: 30
Actual mention total: 60
Actual mention vs Measured science: 1
Actual mention vs Measured mention: 32
Actual mention vs Measured datainfluenced: 3
Actual mention vs Measured other: 0
Actual mention vs Measured zerror: 4
Actual mention vs Measured zlowprob: 18
Actual mention 

Results have been plotted at:
/Users/jamila.pegues/Documents/STScI_Fellowship/Functional/Library/BibTracking/repo_stsci/bibcat/output

Run of evaluate_performance_uncertainty() complete!


And with that, you should have new confusion matrices summarizing the basic performance for these classifiers saved in your requested directory!

---

In [15]:
#Set end marker for this tutorial.
print("This tutorial completed successfully.")

This tutorial completed successfully.
